In [ ]:
from threading import Thread
from time import sleep
import socket
import pickle
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Cipher import PKCS1_v1_5 as Cipher_PKCS1_v1_5
from Crypto.Hash import SHA256
from pprint import pprint

import os
import hashlib

HOST = "192.168.43.3"  # Standard loopback interface address (localhost)
PORT = 9999  # Port to listen on (non-privileged ports are > 1023)

In [ ]:
def generate_RSA_key():
    keys = RSA.generate(2048)
    pub  = RSA.import_key(keys.public_key().export_key('DER'))
    prv  = RSA.import_key(keys.export_key('DER'))
    return prv,pub

def load_RSA_key():
    if(os.path.exists("userkey.pickle")):
        try:
            f=open("userkey.pickle","rb")
            prv,pub = pickle.load(f)
            prv = RSA.import_key(prv)
            pub = RSA.import_key(pub)
            return prv,pub
        except Exception as e:
            print(e)
            return None
    else:
        return None

def save_RSA_key(prv,pub):
    keyfile = open("userkey.pickle","wb")
    pickle.dump([prv.export_key(),pub.export_key()],keyfile)
    keyfile.close()
    print("Write success")
    return True

def encrypt(key, data):
    enc_data = Cipher_PKCS1_v1_5.new(key).encrypt(data)
    return enc_data

def decrypt(key,enc_data):
    '''
    Params
        key - Key for decryption of message
        enc - Encrypted message
    '''
    data = Cipher_PKCS1_v1_5.new(key).decrypt(enc_data,None)
    return data

def sign(keypair,data):
    digest = SHA256.new(data)
    signer = PKCS115_SigScheme(keypair)
    signature = signer.sign(digest)
    return signature


def verify(enc_message,signature,publickey):
    '''
    Params
        enc_message - Encrypted Message to be verified.
        signature - Signature of the message.
        publickey - Public of the Private key used to create signature
    '''
    data,enc_og,pubkey = enc_message,signature,publickey
    digest = SHA256.new(data)
    verifier = PKCS115_SigScheme(publickey)
    try:
        verifier.verify(digest, signature)
        return True
    except:
        return False

def insertDB(dec_message,public_key,torurl):
    global db
    if(torurl not in db):
        db[torurl]={"messages":[],"pubkey":public_key}
    db[torurl]["messages"].append(dec_message)
    
def process_message(data):
    # Format [torurl,enc_message,signature,public_key]
    data = pickle.loads(data)
    torurl,enc_message,signature,public_key = data
    public_key = RSA.import_key(public_key)
    if(verify(enc_message,signature,public_key)):
        dec_message = decrypt(prv,enc_message)
        insertDB(dec_message,public_key,torurl)
        return True
    else:
        print("Verification Failure")
        return False

# Python Server
def receiver_function():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(("0.0.0.0", PORT))
        s.listen()
        while True:
            conn, addr = s.accept()
            with conn:
                # TODO: Threading might give better performance
                while True:
                    # Assuming 1024 bytes as max message size
                    data = conn.recv(1024)
                    if not data:
                        break
                    process_message(data)
    except KeyboardInterrupt:
        print("Bye..")
    
# Client code for the AnonChat server
def send_message(torurl,message):
    HOST = torurl # URL for reciver
    PORT = 9999   # Port for AnonChat on reciever
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        enc_message = encrypt(pub,message.encode())
        signature = sign(prv,enc_message)
        s.send(pickle.dumps(["localhost",enc_message,signature,pub.export_key()]))
        s.close()

In [ ]:
db = {}
keys = load_RSA_key()
prv,pub = (None,None)
if(keys==None):
    print("Keys not avaiable, generating")
    prv,pub = generate_RSA_key()
    print("Saving keys")
    save_RSA_key(prv,pub)
else:
    prv,pub = keys
thread_rec = Thread(target=receiver_function)

In [ ]:
thread_rec.start() # Start main reciver thread

In [ ]:
def UI_Main():
    chosen_person = None
    while(True):
        try:
            inp = input(">").strip()
            if(inp == "$help"):
                print("Help Menu - Conrgatz you have discovered help menu")
                print("1) $menu for menu")
                print("2) $help for help")
                print("Type a message to chat")
            elif(inp == "$menu"):
                print("1) Contact list")
                print("2) Messages")
                print("3) New Contact")
                print("4) Export Public Key")
                print("5) Print DB")
                print("6) Exit")
                inp = input("Choice?[1]:").strip()
                if(inp == "1"):
                    pprint(db.keys())
                    inp = int(input(f"Choice?[{list(db.keys())[0]}]:").strip())
                    chosen_person = list(db.keys())[inp]
                elif(inp == "2"):
                    if(chosen_person):
                        pprint(db[chosen_person]["messages"])
                    else:
                        print("No selected contact, please select one.")
                elif(inp=="3"):
                    inp = input("Contact url:")
                    db[inp]={"messages":[],"pubkey":None}
                    inp = input("PublicKey:").encode()
                    key = None
                    try:
                        key = RSA.import_key(inp)
                    except:
                        print("Import Failure! Please try again!")
                    db[inp]["pubkey"] = key
                elif(inp=="4"):
                    print(pub.export_key("OpenSSH").decode())
                elif(inp=="5"):
                    print(db)
                elif(inp=="6"):
                    break;
                else:
                    print("Try again!")
            else:
                if(chosen_person):
                    send_message(chosen_person,inp)
                else:
                    print("No selected contact, please select one.")
        except KeyboardInterrupt:
            print("Exiting!")
            break
        except Exception as e:
            print(e)
            continue

In [ ]:
UI_Main()

In [ ]:
def client_tester():
    HOST = "localhost"
    PORT = 9999
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        keys = generate_RSA_key() # Priv,Pub
        message = "Hello MOON!"
        enc_message = encrypt(pub,message.encode())
        signature = sign(keys[0],enc_message)
        s.send(pickle.dumps(["localhost",enc_message,signature,keys[1].export_key()]))
        s.close()

In [ ]:
# client_tester()

In [ ]:
db